In [1]:
import mt_preprocess
import h5py
import numpy as np
import example
from tqdm import tqdm
import math
from math import exp

In [2]:
file = h5py.File('../../data/mtsplice/seq.h5','w')

for set in ['test','valid','train']:
    seq_cache=[]
    target_cache=[]
    mean_cache = []
    data_loader = mt_preprocess.Ascot('../../data/mtsplice/gtex_'+set+'_psi.csv',
                                    '/home/ztang/ref/hg38/hg38.fa',
                                    length = 400,
                                    tissues=mt_preprocess.tissues,
                                    encode=True,
                                    pad_trim_same_l=False,
                                    flanking=300,
                                    flanking_exons=False,
                                    region_anno=False,
                                    seq_align='both',
                                    mean_inpute=False,
                                    use_logit=True)
    len = data_loader.__len__()
    for i in tqdm(range(len)):
        item = data_loader.__getitem__(i)
        target = item[1]
        seq = np.concatenate((item[0]['seql'],item[0]['seqr']))
        seq_cache.append(seq)
        target_cache.append(target)
        mean_cache.append(item[0]['mean'])

    file.create_dataset('x_'+set, data= np.array(seq_cache))
    file.create_dataset('y_'+set, data = np.array(target_cache))
    file.create_dataset('m_'+set, data = np.array(mean_cache))

file.close()


100%|██████████| 38028/38028 [00:15<00:00, 2474.24it/s]


In [6]:
file = h5py.File('../../data/mtsplice/delta_logit.h5','w')

for set in ['test','valid','train']:
    seq_cache=[]
    target_cache=[]
    data_loader = mt_preprocess.Ascot('../../data/mtsplice/gtex_'+set+'_psi.csv',
                                    '/home/ztang/ref/hg38/hg38.fa',
                                    length = 400,
                                    tissues=mt_preprocess.tissues,
                                    encode=True,
                                    pad_trim_same_l=False,
                                    flanking=300,
                                    flanking_exons=False,
                                    region_anno=False,
                                    seq_align='both',
                                    mean_inpute=False,
                                    use_logit=True)
    len = data_loader.__len__()
    for i in tqdm(range(len)):
        item = data_loader.__getitem__(i)
        target = item[1]
        mean = item[0]['mean']
        target = target - mean
        target_m = np.vstack((target,mean))
        seq = np.concatenate((item[0]['seql'],item[0]['seqr']))
        seq_cache.append(seq)
        target_cache.append(target_m.T)

    file.create_dataset('x_'+set, data= np.array(seq_cache))
    file.create_dataset('y_'+set, data = np.array(target_cache))

file.close()


100%|██████████| 38028/38028 [00:15<00:00, 2436.26it/s]
